In [190]:
#importing modules 
import json 
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process

# test file path
path = "./test_directory_files/"
test_json = "119.56837603.5abc93e0-6e04-0134-173e-00505686a51c_labeled.json"
street_names_txt = "./city-directory-entry-parser-master/streetnames.txt"

c:\users\wesch\appdata\local\programs\python\python37\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


[Link](https://stackoverflow.com/questions/40712178/reading-the-json-file-with-multiple-objects-in-python) to help with decoding this particular JSON object. Multiple JSON object within one.

In [2]:
# Opens the file and saves it to a pandas df
with open(path + test_json) as f:
    df = pd.read_json(f, lines=True)

In [73]:
# Quick look into the JSON table to check
df.tail()

,directory_uuid,page_uuid,entry_uuid,total_lines_from_hocr,original_hocr_line_number,bbox,col,appended,skipped_line_after,complete_entry,labeled_entry
147,1889.4b939190-317a-0134-d1d5-00505686a51c,119.56837603.5abc93e0-6e04-0134-173e-00505686a51c,9e73cd7a40ec11eabf9be0accb5fbfb2,196,193,814 3252 1579 3304,2,no,0,"Beahling Charles, tracks, 24 Beaver, h 218 Cherry","{'subjects': ['Beahling Charles'], 'occupation..."
148,1889.4b939190-317a-0134-d1d5-00505686a51c,119.56837603.5abc93e0-6e04-0134-173e-00505686a51c,9e73ce6240ec11ea8acae0accb5fbfb2,196,194,814 3301 1377 3334,2,no,0,"Beahre Daniel, teacher, h 847 E. 81st","{'subjects': ['Beahre Daniel'], 'occupations':..."
149,1889.4b939190-317a-0134-d1d5-00505686a51c,119.56837603.5abc93e0-6e04-0134-173e-00505686a51c,9e73cf6e40ec11eab923e0accb5fbfb2,196,195,814 3322 1578 3364,2,no,0,"Beakes Albert 8. milk, 307 W. 44th, h 347 W. 34th","{'subjects': ['Beakes Albert 8'], 'occupations..."
150,1889.4b939190-317a-0134-d1d5-00505686a51c,119.56837603.5abc93e0-6e04-0134-173e-00505686a51c,9e73d04a40ec11eab7f0e0accb5fbfb2,196,196,814 3366 1575 3397,2,yes,0,"Beakes Charles H. CO. milk, 206 E. 12th & 307 ...","{'subjects': ['Beakes Charles H. CO'], 'occupa..."
151,1889.4b939190-317a-0134-d1d5-00505686a51c,119.56837603.5abc93e0-6e04-0134-173e-00505686a51c,9e73d18a40ec11eaa8fde0accb5fbfb2,196,198,813 3431 1378 3464,2,no,0,"Beakes A. 8. & Co. milk, 307 W. 44th","{'subjects': ['Beakes A'], 'occupations': ['mi..."


In [41]:
# Manual uniqueness check
line_number = np.array(df.original_hocr_line_number)
unique_elements, counts_elements = np.unique(line_number, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[  6   7   8   9  11  13  15  16  17  18  20  21  22  23  24  26  27  28
   29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  46  47
   48  49  51  52  53  54  55  56  57  58  59  60  62  63  64  65  67  68
   69  70  71  72  73  74  75  76  77  78  79  80  82  83  84  85  86  87
   88  89  90  91  92  93  94  95  97  98  99 100 103 104 109 117 119 120
  121 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139
  140 141 142 143 144 148 149 150 151 152 154 155 156 157 158 159 161 162
  163 164 165 166 167 169 171 172 173 174 176 178 180 184 186 187 188 189
  190 191 192 193 194 195 196 198]
 [  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
    1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
    1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
    1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
    1   1   1   1   1   1   1  

Labels "a", "h", and "r" represent the relative addresses, and address types such as houses, and rear.

In [13]:
# Quick glimpse of how the last column might actually look like
df.iloc[124,-1]

{'subjects': ['Beacom Thomas W'],
 'occupations': ['painter'],
 'locations': [{'value': '301 W. 127th', 'labels': ['h']}]}

In [11]:
df.iloc[110,-1]

{'subjects': ['Beach Warren C'],
 'occupations': ['u, s'],
 'locations': [{'value': '585 Fifth av,', 'labels': ['a', ' h']}]}

In [12]:
df.iloc[10,-1]

{'subjects': ['Bayot Emil'],
 'occupations': ['jeweler'],
 'locations': [{'value': '103 B. 14th'}]}

In [19]:
df.iloc[4,-1]

{'subjects': ['Baynes ee'],
 'occupations': ['sec'],
 'locations': [{'value': '523 W. 24th'},
  {'value': "8. Boulevard n Hunt's Point rd.", 'labels': ['h']}]}

In [21]:
df.iloc[147, -1]

{'subjects': ['Beahling Charles'],
 'occupations': ['tracks'],
 'locations': [{'value': '24 Beaver'},
  {'value': '218 Cherry', 'labels': ['h']}]}

In [82]:
df_labeled_entry = df.labeled_entry.apply(pd.Series)
df_locations = df_labeled_entry["locations"]
df_locations = pd.concat([df_locations, df.original_hocr_line_number], axis = 1)

In [83]:
df_locations

,locations,original_hocr_line_number
0,[],6
1,[{'value': '89 Front'}],7
2,[{'value': '89 Front'}],8
3,"[{'value': '105 Franklin'}, {'value': '123 E, ...",9
4,"[{'value': '523 W. 24th'}, {'value': '8. Boule...",11
...,...,...
147,"[{'value': '24 Beaver'}, {'value': '218 Cherry...",193
148,"[{'value': '847 E. 81st', 'labels': ['h']}]",194
149,"[{'value': '307 W. 44th'}, {'value': '347 W. 3...",195
150,"[{'value': '206 E. 12th & 307 W. 44th'}, {'val...",196


In [59]:
df_locations.iloc[147,:]

locations                    [{'value': '24 Beaver'}, {'value': '218 Cherry...
original_hocr_line_number                                                  193
Name: 147, dtype: object

In [60]:
type(df_locations.iloc[147,0])

list

In [61]:
type(df_locations.iloc[147,0][1])

dict

In [63]:
df_locations.iloc[147][0]

[{'value': '24 Beaver'}, {'value': '218 Cherry', 'labels': ['h']}]

In other words, each labeled entry's location is stored as a list of _n_ dictionaries with two key-value pairs of `value` and `labels`.

In [65]:
df_locations.iloc[3][0]

[{'value': '105 Franklin'}, {'value': '123 E, 5%!', 'labels': ['h']}]

In [70]:
type(df_locations)

pandas.core.frame.DataFrame

In [186]:
# This checks for existence of labels, duly records the (non)existence, strips it out, appends to another list
# This is done w/o loss to order

line_num_list = []
complete_street_name = []
split_street_name = []
labels = []
for entry in range(0,len(df_locations)):
    #print(df_locations.iloc[entry])
    for dicts in df_locations.iloc[entry][0]:
        if 'labels' in dicts:
            for key, value in dicts.items():
                #line_num_list.append(df_locations.iloc[entry][1])
                if key == 'labels':
                    labels.append(value)
                else:
                    complete_street_name.append(value)
                    street_name_split = str(value).split()
                    split_street_name.append(street_name_split)
                    line_num_list.append(df_locations.iloc[entry][1])
        else:
            for key, value in dicts.items():
                line_num_list.append(df_locations.iloc[entry][1])
                labels.append(" ")
                complete_street_name.append(value)
                street_name_split = str(value).split()
                split_street_name.append(street_name_split)

In [121]:
del unlisted_location

It appears that avenues are spelled out and streets are enumerated.

In [179]:
split_street_name

[['89', 'Front'],
 ['89', 'Front'],
 ['105', 'Franklin'],
 ['123', 'E,', '5%!'],
 ['523', 'W.', '24th'],
 ['8.', 'Boulevard', 'n', "Hunt's", 'Point', 'rd.'],
 ['1634', "B'way"],
 ["Hunt's", 'Point'],
 ['523', 'W.', '24th'],
 ['534', 'W.', '4%th'],
 ['155', 'E.', '21st'],
 ['89', 'Reade'],
 ['Plainfield,', 'N.', 'J.'],
 ['103', 'B.', '14th'],
 ['3', 'Chambers'],
 ['54', 'Barclay', '&', '182', 'Cherry'],
 ['900', 'Second', 'av.'],
 ['245', 'Biway'],
 ['117', 'E.', 'Tith'],
 ['78', 'Orchard'],
 ['78', 'Orchard'],
 ['231', 'E.', '38th'],
 ['76', 'E.', '113th', '-'],
 ['Maker', '(periodical)', '46', 'E.', '14th'],
 ['415', 'Fifth'],
 ['344', 'E.', '74th'],
 ['167', 'Ludlow'],
 ['243', 'BE.', '33d'],
 ['183', 'W.', '68d'],
 ['62', 'Tenth', 'av.'],
 ['9', 'Sixth', 'av.'],
 ['363', 'E.', '71st'],
 ['404', 'W.', '54th'],
 ['41', 'Wall'],
 ['825', 'Sixth', 'av'],
 ['62', 'S.', '5th', 'av.'],
 ['§25', 'Sixth', 'av.'],
 ['Woodlawn'],
 ['203', 'E.', '88th'],
 ['157', 'E.', '86th'],
 ['157', 'E.', '

In [188]:
unlisted_location = pd.concat([pd.Series(complete_street_name), pd.Series(split_street_name), pd.Series(labels), pd.Series(line_num_list)], axis = 1)
unlisted_location.columns = ["complete_street_name", "split_street_name", "labels", "line_number"]

In [187]:
len(line_num_list)

198

In [189]:
unlisted_location

,complete_street_name,split_street_name,labels,line_number
0,89 Front,"[89, Front]",,7
1,89 Front,"[89, Front]",,8
2,105 Franklin,"[105, Franklin]",,9
3,"123 E, 5%!","[123, E,, 5%!]",[h],9
4,523 W. 24th,"[523, W., 24th]",,11
...,...,...,...,...
193,347 W. 34th,"[347, W., 34th]",[h],195
194,206 E. 12th & 307 W. 44th,"[206, E., 12th, &, 307, W., 44th]",,196
195,"Grand Central h,","[Grand, Central, h,]",[h],196
196,8. & Co,"[8., &, Co]",,198


In [202]:
# first order of business is to test if string is a street number
# if it is not, then it is probably a street name, and a comparison can be made
# Perform regex to remove the periods
# https://www.datacamp.com/community/tutorials/fuzzy-string-python
# https://datascience.stackexchange.com/questions/29775/how-to-check-and-correct-misspelling-in-the-data-of-pairs-of-words
# http://blog.keyrus.co.uk/fuzzy_matching_101_part_i.html
# http://blog.keyrus.co.uk/fuzzy_matching_101_part_ii.html

for i in range(0, len(unlisted_location)):
    non_numeric_street_name = []
    for address_component in unlisted_location.iloc[i, 1]:
        if address_component.isnumeric() == False:
            non_numeric_street_name.append(address_component)
            print(non_numeric_street_name)
        unlisted_location.iloc[i]["non_numeric"] = non_numeric_street_name
            
        #print(address_component.isnumeric())

['Front']
['Front']
['Franklin']
['E,']
['E,', '5%!']
['W.']
['W.', '24th']
['8.']
['8.', 'Boulevard']
['8.', 'Boulevard', 'n']
['8.', 'Boulevard', 'n', "Hunt's"]
['8.', 'Boulevard', 'n', "Hunt's", 'Point']
['8.', 'Boulevard', 'n', "Hunt's", 'Point', 'rd.']
["B'way"]
["Hunt's"]
["Hunt's", 'Point']
['W.']
['W.', '24th']
['W.']
['W.', '4%th']
['E.']
['E.', '21st']
['Reade']
['Plainfield,']
['Plainfield,', 'N.']
['Plainfield,', 'N.', 'J.']
['B.']
['B.', '14th']
['Chambers']
['Barclay']
['Barclay', '&']
['Barclay', '&', 'Cherry']
['Second']
['Second', 'av.']
['Biway']
['E.']
['E.', 'Tith']
['Orchard']
['Orchard']
['E.']
['E.', '38th']
['E.']
['E.', '113th']
['E.', '113th', '-']
['Maker']
['Maker', '(periodical)']
['Maker', '(periodical)', 'E.']
['Maker', '(periodical)', 'E.', '14th']
['Fifth']
['E.']
['E.', '74th']
['Ludlow']
['BE.']
['BE.', '33d']
['W.']
['W.', '68d']
['Tenth']
['Tenth', 'av.']
['Sixth']
['Sixth', 'av.']
['E.']
['E.', '71st']
['W.']
['W.', '54th']
['Wall']
['Sixth']
['Six

c:\users\wesch\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [203]:
unlisted_location

,complete_street_name,split_street_name,labels,line_number,non_numeric
0,89 Front,"[89, Front]",,7,NaN
1,89 Front,"[89, Front]",,8,NaN
2,105 Franklin,"[105, Franklin]",,9,NaN
3,"123 E, 5%!","[123, E,, 5%!]",[h],9,NaN
4,523 W. 24th,"[523, W., 24th]",,11,NaN
...,...,...,...,...,...
193,347 W. 34th,"[347, W., 34th]",[h],195,NaN
194,206 E. 12th & 307 W. 44th,"[206, E., 12th, &, 307, W., 44th]",,196,NaN
195,"Grand Central h,","[Grand, Central, h,]",[h],196,NaN
196,8. & Co,"[8., &, Co]",,198,NaN


In [176]:
master_street_names = pd.read_csv(street_names_txt, header = None)

In [177]:
master_street_names

,0
0,Fourth
1,Gold
2,Congress
3,Dominick
4,Washington
...,...
450,57th
451,Fifty-Eighth
452,58th
453,Fifty-Ninth
